# Lab 03: AIP with Boto3 - Multi-Tenant Marketing Platform

## Business Context
You are building a **Multi-Tenant Marketing Platform** that serves multiple enterprise clients. Your platform generates AI-powered marketing campaigns for different tenants:

- **Tenant A**: B2B tech company needing product launch campaigns
- **Tenant B**: B2C retail company needing promotional campaigns

**Challenge**: Track model usage, costs, and performance per client for accurate billing and SLA monitoring.

## Learning Objectives
- Implement Application Inference Profiles using boto3
- Create tenant-specific AIP configurations
- Generate marketing campaigns with tenant isolation
- Track usage and costs per client for billing

In [ ]:
# Install required packages
!pip install --force-reinstall -r requirements.txt --quiet

## Section 1: Setup and Create Application Inference Profiles

First, let's set up our boto3 client and create Application Inference Profiles for our two tenants.

In [ ]:
import boto3
import json
import time
from datetime import datetime
from typing import Dict, Any
from lab_helpers.config import Region, ModelId
from lab_helpers.aip_manager import AIPManager
from lab_helpers.usage_tracker import UsageTracker

# Initialize AWS clients
bedrock_client = boto3.client('bedrock', region_name=Region)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=Region)
sts = boto3.client('sts', region_name=Region)

print(f"✅ Initialized boto3 clients for region: {Region}")
print(f"📋 Using Claude Sonnet 4 model: {ModelId}")

  ### 🏗️ Setup: AIP Manager and Tenant Configurations

  Initialize the Application Inference Profile manager and define configurations for two tenants:
  - **Tenant A (B2B Tech)**: SaaS DevOps platform marketing
  - **Tenant B (B2C Retail)**: Fashion e-commerce marketing

  Each tenant gets:
  - Unique AIP for isolated model access
  - Tags for cost tracking and billing (`TenantId`, `BusinessType`, `CostCenter`)
  - Separate CloudWatch metrics dimensions

In [ ]:
# Initialize AIP Manager
aip_manager = AIPManager(bedrock_client)

# Define tenant configurations
TENANT_CONFIGS = {
    "tenant_a": {
        "name": "marketing-ai-tenant-a",
        "description": "Marketing AI AIP for Tenant A",
        "tags": {
            "TenantId": "tenant-a",
            "BusinessType": "B2B-Tech",
            "Environment": "production",
            "CostCenter": "marketing-ai-platform"
        }
    },
    "tenant_b": {
        "name": "marketing-ai-tenant-b", 
        "description": "Marketing AI AIP for Tenant B",
        "tags": {
            "TenantId": "tenant-b",
            "BusinessType": "B2C-Retail",
            "Environment": "production",
            "CostCenter": "marketing-ai-platform"
        }
    }
}

print("📋 Tenant configurations defined:")
for tenant_id, config in TENANT_CONFIGS.items():
    print(f"  - {tenant_id}: {config['description']}")

### 🔧 Create Application Inference Profiles for Each Tenant

  This cell creates (or verifies existing) Application Inference Profiles for both tenants using the AWS Bedrock API.

  **The AIP Creation Process:**

  1. **Check for Existing AIPs**: Verify if tenant already has an AIP to avoid duplicates
  2. **Prepare Tags**: Convert tenant tags to AWS API format for cost allocation
  3. **Construct Model ARN**: Build proper ARN pointing to the base System Inference Profile
  4. **Create AIP**: Call `create_inference_profile()` with:
     - Unique profile name per tenant
     - Model source (copies from System Inference Profile)
     - Tenant-specific tags for tracking

  **🎯 The Critical API Call:**
  ```python
  bedrock_client.create_inference_profile(
      inferenceProfileName=config["name"],
      description=config["description"],
      modelSource={"copyFrom": MODEL_ARN},
      tags=tag_list
  )

  What you get:
  - Unique AIP ARN for each tenant
  - Isolated CloudWatch metrics dimension (ModelId = AIP ARN)
  - Tagged resources for cost allocation and billing

  Result: Two Application Inference Profiles ready for tenant-isolated model inference!

In [ ]:
# Create Application Inference Profiles for both tenants
tenant_aips = {}

for tenant_id, config in TENANT_CONFIGS.items():
    print(f"\n🔄 Creating AIP for {tenant_id}...")
    
    try:
        # Check if AIP already exists
        existing_arn = aip_manager.check_aip_exists(config["name"])
        
        if existing_arn:
            print(f"✅ AIP already exists for {tenant_id}")
            tenant_aips[tenant_id] = existing_arn
        else:
            # Prepare tags in the format expected by AWS API
            tag_list = []
            if config["tags"]:
                tag_list = [{"key": k, "value": v} for k, v in config["tags"].items()]
            
            # Get account ID
            account_id = sts.get_caller_identity()['Account']
            print(f"Account ID: {account_id}")

            # Construct proper ARN
            MODEL_ARN = f"arn:aws:bedrock:{Region}:{account_id}:inference-profile/{ModelId}"
            print(f"Model ARN: {MODEL_ARN}")

            # 🎯 CREATE APPLICATION INFERENCE PROFILE - This is the core AIP API call!
            response = bedrock_client.create_inference_profile(
                inferenceProfileName=config["name"],
                description=config["description"],
                modelSource={
                    "copyFrom": MODEL_ARN  # Use proper ARN format
                },
                tags=tag_list
            )
            
            aip_arn = response['inferenceProfileArn']
            tenant_aips[tenant_id] = aip_arn
            
            print(f"✅ Created AIP for {tenant_id}")
            print(f"   Status: {response['status']}")
        
        print(f"   ARN: {tenant_aips[tenant_id]}")
        
    except Exception as e:
        print(f"❌ Error creating AIP for {tenant_id}: {str(e)}")

print(f"\n📊 Summary: {len(tenant_aips)} Application Inference Profiles ready")

### 🔍 Key Differences: System vs Application Inference Profiles

Let's examine what we just created and how it differs from System Inference Profiles:

In [ ]:
# Compare System vs Application Inference Profile usage
print("🔍 SYSTEM vs APPLICATION INFERENCE PROFILES")
print("=" * 55)

print("\n📋 SYSTEM INFERENCE PROFILE (Traditional):")
print(f"   Model ID: {ModelId}")
print("   Usage Tracking: ❌ No tenant-level visibility")
print("   Cost Allocation: ❌ Shared across all tenants")
print("   Billing: ❌ Cannot separate by tenant")
print("   Monitoring: ❌ Aggregated metrics only")

print("\n📋 APPLICATION INFERENCE PROFILES (Multi-tenant):")
for tenant_id, aip_arn in tenant_aips.items():
    print(f"\n   🏢 {tenant_id.upper()}:")
    print(f"      AIP ARN: {aip_arn}")
    print("      Usage Tracking: ✅ Per-tenant visibility")
    print("      Cost Allocation: ✅ Isolated per tenant")
    print("      Billing: ✅ Accurate tenant billing")
    print("      Monitoring: ✅ Tenant-specific metrics")

print("\n💡 KEY INSIGHT: AIPs enable precise tenant isolation while using the same underlying model!")

### 📊 CloudWatch Monitoring Function

  The `monitor_aip_usage()` function fetches and visualizes CloudWatch metrics for each tenant's Application Inference Profile:

  **What it does:**
  - Queries CloudWatch for the last 60 minutes of data
  - Fetches three key metrics per tenant:
    - **Invocations**: Number of API calls
    - **InputTokenCount**: Request complexity
    - **OutputTokenCount**: Response generation
  - Generates time-series plots showing usage patterns
  - Returns metrics dictionary for cost analysis

  **Why it matters:**
  - Each tenant's metrics are **isolated** using their unique AIP ID
  - Enables accurate per-tenant billing and cost allocation
  - Provides proof that multi-tenancy isolation is working

  **Key insight:** The AIP ARN's last segment becomes the CloudWatch `ModelId` dimension, enabling separate tracking per tenant.

In [ ]:
def monitor_aip_usage(tenant_aips, region):
    """
    Monitor CloudWatch metrics for multiple Application Inference Profiles.
    
    Args:
        tenant_aips (dict): Mapping of tenant_id -> AIP ARN
        region (str): AWS region where AIPs are deployed
    
    Returns:
        dict: Tenant metrics for further analysis
    """
    # Import CloudWatch helper functions for Bedrock AIP monitoring
    # fetch_metrices: Retrieves Invocations, InputTokenCount, OutputTokenCount from AWS/Bedrock namespace
    # plot_graph: Creates time-series visualizations for usage pattern analysis
    from lab_helpers.cloudwatch import fetch_metrices, plot_graph

    # Display monitoring session information
    print("📊 Fetching CloudWatch metrics for Application Inference Profiles...")
    print(f"Region: {region}")
    print(f"Time Range: Last 60 minutes")  # Fixed 60-minute window for consistent analysis
    print("="*80)

    # Initialize storage for cross-tenant metrics comparison
    # This enables comparative analysis between different tenant usage patterns
    tenant_metrics = {}

    # Iterate through each tenant's Application Inference Profile
    # tenant_aips contains mapping: tenant_id -> AIP ARN
    for tenant_id, aip_arn in tenant_aips.items():
        print(f"\n🏢 TENANT: {tenant_id.upper()}")
        print("="*50)
        print(f"AIP ARN: {aip_arn}")
        
        # Extract AIP ID from full ARN for CloudWatch ModelId dimension
        # CloudWatch requires just the ID (last part), not the complete ARN
        # Example: arn:aws:bedrock:us-west-2:123456789012:inference-profile/tenant-a-claude-sonnet
        #          becomes: tenant-a-claude-sonnet
        aip_id = aip_arn.split('/')[-1]
        print(f"AIP ID: {aip_id}")
        
        try:
            # Fetch real-time CloudWatch metrics for this specific tenant's AIP
            # This provides tenant-isolated usage tracking for cost allocation
            print(f"\n📊 METRICS FOR {tenant_id.upper()}:")
            response, input_token_response, output_token_response = fetch_metrices(
                Region=region,      # AWS region where AIP is deployed
                Period=60,          # 1-minute aggregation periods (parameter currently unused)
                Timedelta=60,       # 60-minute time window (parameter currently unused)
                Id=aip_id          # AIP ID for CloudWatch ModelId dimension filtering
            )
            
            # Store metrics for cross-tenant comparison and analysis
            # Enables cost allocation, usage pattern analysis, and capacity planning
            tenant_metrics[tenant_id] = {
                'invocations': response,              # API call frequency data
                'input_tokens': input_token_response, # Request complexity metrics
                'output_tokens': output_token_response # Response generation metrics
            }
            
            # Generate time-series visualization for this tenant
            # Shows usage patterns, peak times, and trend analysis
            print(f"\n📈 USAGE PLOTS FOR {tenant_id.upper()}:")
            plot_graph(response, input_token_response, output_token_response)
            
            print("="*50)
                
        except Exception as e:
            # Handle CloudWatch API errors gracefully
            # Common causes: insufficient permissions, no data available, API throttling
            print(f"⚠️ CloudWatch error for {tenant_id}: {str(e)}")
            print(f"💡 Metrics may take a few minutes to appear after model invocations")
            print("="*50)
    
    return tenant_metrics

## 📊 Baseline Check: CloudWatch Metrics BEFORE Campaign Generation

Before we generate any campaigns, let's check CloudWatch metrics for our newly created AIPs. This establishes a **baseline** that proves:

1. ✅ The AIPs exist and are properly configured
2. ✅ Each tenant has their own isolated CloudWatch dimension
3. ✅ No usage has occurred yet (metrics should be empty)

### The Before/After Demonstration

We'll check metrics **twice** in this lab:

**🔵 NOW (BEFORE):** Check metrics → Expect empty/no data
- This proves the AIPs are ready but haven't been used

**🟢 LATER (AFTER):** Check metrics → Expect populated per-tenant data  
- This proves multi-tenant isolation is working!

Let's run the baseline check:

In [ ]:
# Check CloudWatch metrics BEFORE any campaign generation
print("🔵 BEFORE CHECK: Querying CloudWatch for baseline metrics...")
print("="*80)
print("Expected: No metrics or empty data (AIPs exist but unused)")
print("="*80 + "\n")

baseline_metrics = monitor_aip_usage(tenant_aips, Region)

print("\n" + "="*80)
print("✅ Baseline check complete!")
print("💡 Note: If you see 'No metrics data found', that's expected!")
print("   The AIPs are ready, but we haven't made any inferences yet.")
print("="*80)

## Section 2: Tenant-Aware Marketing Campaign Generation

### 🤖 Marketing Campaign Generation Function

  Generates AI-powered marketing campaigns using tenant-specific Application Inference Profiles.

  **The Multi-Tenant Pattern:**
  ```python
  bedrock_runtime.invoke_model(
      modelId=aip_arn,  # ← AIP ARN instead of model ID
      body=json.dumps(request_body)
  )

  What happens:
  1. Accept tenant ID, AIP ARN, and campaign brief
  2. Build prompt requesting marketing campaign
  3. Invoke Claude using AIP ARN (not base model ID)
  4. Track usage metrics: input tokens, output tokens, latency
  5. Return campaign content with tenant-tagged metrics

  The Result: Each tenant's usage automatically tracked in CloudWatch under their unique AIP dimension!

In [ ]:
def generate_marketing_campaign(tenant_id: str, aip_arn: str, campaign_brief: str) -> Dict[str, Any]:
    """
    Generate marketing campaign using tenant-specific AIP
    
    Args:
        tenant_id: Tenant identifier
        aip_arn: Application Inference Profile ARN
        campaign_brief: Campaign requirements
        
    Returns:
        Campaign response and usage metrics
    """
    
    # Prepare the prompt
    prompt = f"""
You are a marketing AI assistant. Generate a comprehensive marketing campaign based on the following brief:

{campaign_brief}

Please provide:
1. Campaign Theme & Messaging
2. Target Audience Analysis
3. Key Marketing Channels
4. Content Recommendations
5. Success Metrics

Keep the response concise but actionable.
"""
    
    # Prepare request body
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    
    try:
        # Record start time for latency calculation
        start_time = datetime.now()
        
        # Invoke model using AIP
        response = bedrock_runtime.invoke_model(
            modelId=aip_arn,  # Use AIP ARN instead of model ID
            body=json.dumps(request_body),
            contentType="application/json",
            accept="application/json"
        )
        
        # Calculate latency
        end_time = datetime.now()
        latency_ms = (end_time - start_time).total_seconds() * 1000
        
        # Parse response
        response_body = json.loads(response['body'].read())
        
        # Extract campaign content
        campaign_content = response_body['content'][0]['text']
        
        # Extract usage metrics
        usage_metrics = {
            'input_tokens': response_body.get('usage', {}).get('input_tokens', 0),
            'output_tokens': response_body.get('usage', {}).get('output_tokens', 0),
            'latency_ms': latency_ms
        }
        
        return {
            'tenant_id': tenant_id,
            'aip_arn': aip_arn,
            'campaign_content': campaign_content,
            'usage_metrics': usage_metrics,
            'timestamp': datetime.now().isoformat()
        }
        
    except Exception as e:
        return {
            'tenant_id': tenant_id,
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        }

print("✅ Marketing campaign generation function defined")

### 📋 Campaign Briefs for Multi-Tenant Demo

  Define marketing campaign specifications for two tenant profiles:
  - **Tenant A**: B2B SaaS DevOps platform (lead generation)
  - **Tenant B**: B2C fashion retail (sales growth)

  Each campaign will be generated using tenant-specific AIPs for isolated tracking.

In [ ]:
# Define campaign briefs for each tenant
CAMPAIGN_BRIEFS = {
    "tenant_a": """
Campaign Brief for B2B Tech Company:
- Product: New SaaS platform for DevOps automation
- Target: CTOs and Engineering Directors at mid-size companies (100-1000 employees)
- Goal: Generate 500 qualified leads for product demo
- Budget: $50,000 over 3 months
- Key Message: "Reduce deployment time by 80% with AI-powered DevOps"
""",
    "tenant_b": """
Campaign Brief for B2C Retail Company:
- Product: Summer fashion collection for millennials
- Target: Fashion-conscious millennials aged 25-35 with disposable income
- Goal: Increase online sales by 25% during summer season
- Budget: $75,000 over 2 months
- Key Message: "Sustainable fashion that doesn't compromise on style"
"""
}

print("📋 Campaign briefs defined for both tenants")

### 🎯 Generate Marketing Campaigns for Each Tenant

  This cell executes AI-powered campaign generation for both tenants:

  **What happens:**
  1. Iterates through each tenant configuration
  2. Calls Claude via their tenant-specific Application Inference Profile
  3. Generates customized marketing campaigns based on their briefs
  4. Tracks generation metrics (tokens, latency) for each tenant
  5. Stores results for display in the next section

  **Key insight:** Each tenant's request uses their isolated AIP, enabling separate usage tracking and billing in CloudWatch.

In [ ]:
# Generate campaigns for both tenants
campaign_results = {}

for tenant_id in TENANT_CONFIGS.keys():
      if tenant_id in tenant_aips:
          print(f"\n🎯 Generating campaign for {tenant_id}...")

          result = generate_marketing_campaign(
              tenant_id=tenant_id,
              aip_arn=tenant_aips[tenant_id],
              campaign_brief=CAMPAIGN_BRIEFS[tenant_id]
          )

          campaign_results[tenant_id] = result

          if 'error' not in result:
              print(f"✅ Campaign generated for {tenant_id}")
              print(f"   Input tokens: {result['usage_metrics']['input_tokens']}")
              print(f"   Output tokens: {result['usage_metrics']['output_tokens']}")
              print(f"   Latency: {result['usage_metrics']['latency_ms']:.2f}ms")
          else:
              print(f"❌ Error generating campaign for {tenant_id}: {result['error']}")

print(f"\n📊 Generated campaigns for {len(campaign_results)} tenants")

## Section 3: Review Generated Marketing Campaigns

## 📊 CloudWatch Metrics Check: After Campaign Generation

  ### 🟢 The "AFTER" State - Proving Multi-Tenant Isolation

  Now that we've generated campaigns using tenant-specific AIPs, let's check CloudWatch metrics to see the results.

  **What we expect to see:**

  **✅ Per-Tenant Metrics Populated:**
  - **Tenant A**: Separate invocations, input tokens, output tokens
  - **Tenant B**: Separate invocations, input tokens, output tokens
  - **Visual Proof**: Distinct graphs for each tenant (not combined!)

  **⏳ Why the 60-second wait?**
  CloudWatch metrics have a propagation delay of 1-2 minutes. We wait 60 seconds to ensure metrics from our campaign generation
  are available for querying.

  **The Before/After Proof:**

  | Metric Check | Expected Result | What It Proves |
  |--------------|----------------|----------------|
  | **BEFORE** (earlier) | Empty/no data | AIPs exist but unused |
  | **AFTER** (now) | Populated per-tenant data | Multi-tenant isolation works! |

  **💡 What makes this powerful:**
  - Same underlying Claude model
  - Same API calls (`invoke_model`)
  - **Different tracking** - Each tenant's usage isolated in CloudWatch
  - Enables accurate billing, SLA monitoring, and cost allocation

  Let's fetch the metrics and visualize the proof:

In [ ]:
# Check CloudWatch metrics After  campaign generation
import time
print("⏳ Waiting 60 seconds for CloudWatch metrics to propagate...")
time.sleep(60)
print("✅ Wait complete - proceeding with metrics monitoring")

baseline_metrics = monitor_aip_usage(tenant_aips, Region)

print("\n" + "="*80)
print("✅ Baseline check complete!")
print("💡 Note: If you see 'No metrics data found', that's expected!")
print("   The AIPs are ready, but we haven't made any inferences yet.")
print("="*80)

## 🟢 AFTER Check: CloudWatch Metrics Show Multi-Tenant Isolation!

Now that we've generated campaigns using tenant-specific AIPs, let's check CloudWatch metrics again. This time, you should see:

### Expected Results:

**📊 Per-Tenant Metrics:**
- **Tenant A** metrics under their own AIP dimension
- **Tenant B** metrics under their own AIP dimension
- **Separate graphs** for each tenant (not combined!)

### The Proof of Multi-Tenancy:

Compare this to the BEFORE check:
- **BEFORE:** Empty metrics (AIPs ready but unused)
- **AFTER:** Populated metrics showing exact per-tenant usage

This demonstrates:
1. ✅ **Tenant Isolation**: Each tenant's usage tracked separately
2. ✅ **Accurate Billing**: Can charge tenant_a and tenant_b independently
3. ✅ **SLA Monitoring**: Can track performance per customer
4. ✅ **Cost Allocation**: Know exactly what each tenant costs you

⏳ **Note:** CloudWatch metrics propagate within 1-2 minutes. If you ran the campaigns just now, wait ~60 seconds and re-run the cell below.

## Lab Summary

🎉 **Congratulations!** You've successfully implemented Application Inference Profiles with boto3 for a multi-tenant Marketing AI platform.

### What You Accomplished:

1. **✅ Created Application Inference Profiles** for two tenants using boto3
2. **✅ Checked CloudWatch metrics BEFORE campaigns** (baseline/empty state)
3. **✅ Generated tenant-specific marketing campaigns** with isolated usage tracking
4. **✅ Displayed AI-generated campaign content** for each tenant
5. **✅ Checked CloudWatch metrics AFTER campaigns** (per-tenant usage visible!)
6. **✅ Proved multi-tenant isolation** through before/after comparison

### Key Takeaways:

- **Tenant Isolation**: Each tenant's model usage is tracked separately through AIPs
- **Cost Allocation**: Usage metrics enable accurate per-tenant billing
- **CloudWatch Integration**: Real-time per-tenant monitoring with separate graphs
- **Production Readiness**: Error handling ensures reliability
- **Business Value**: Enable usage-based pricing and demonstrate ROI

### What You Learned:

```python
# Traditional approach (no tenant visibility)
bedrock_runtime.invoke_model(modelId="us.anthropic.claude-sonnet-4-...")

# AIP approach (full tenant isolation)
bedrock_runtime.invoke_model(modelId=tenant_aip_arn)  # Tracked separately!
```

### The Before/After Pattern:

- **BEFORE Check**: Empty CloudWatch metrics (AIPs ready but unused)
- **AFTER Check**: Populated metrics showing exact per-tenant usage
- **Proof**: Multi-tenant isolation works!

### Next Steps:

Continue your AIP journey with higher-level framework integrations:
- **Lab 04**: Strands Agents - Build tenant-aware agent applications
- **Lab 05**: LiteLLM Gateway - Implement gateway patterns with AIP
- **Lab 06 (Optional)**: LangChain - Chain-based applications with tenant awareness
- **Lab 07 (Optional)**: LangGraph - Graph workflows for complex multi-step processes
- **Lab 08**: AgentCore Runtime - Production deployment with full observability

**Ready to build agent applications?** → [Continue to Lab 04: Strands Agents](lab-04-aip-strands.ipynb)